## Instalação de Pacotes

## Importação de Bibliotecas e Funções

In [ ]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import math
import sys

import matplotlib.pyplot as plt
import seaborn as sns

# Efetuar leituras do Yahoo Finance
from pandas_datareader.data import DataReader
import yfinance as yf

# Para as Datas das Ações
from datetime import datetime

yf.pdr_override()

sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")
%matplotlib inline

In [ ]:
def str_to_class(classname):
    return getattr(sys.modules[__name__], classname)


def stock_dicts(stocks_list, start, end):
    company_dicts = []
    company_list = []
    dataframes = []

    for company, stock in tqdm(stocks_list.items()):
        if "-" in stock:
            stock_name = stock.replace("-", "_")
        else:
            stock_name = stock
        globals()[stock_name] = yf.download(stock, start, end, progress=False)
        company_list.append(company)
        company_dicts.append(stock_name)

    for company_dict, company_name in zip(company_dicts, company_list):
        data = str_to_class(company_dict)
        data["company_name"] = company_name
        dataframes.append(data)
    df = pd.concat(dataframes, axis=0)

    return df, company_list, company_dicts


def fechamento_ano(stock_list):
    _, _, company_dicts = stock_dicts(
        stock_list,
        datetime(
            (datetime.now()).year - 1, (datetime.now()).month, (datetime.now()).day
        ),
        datetime.now(),
    )

    plt.figure(constrained_layout=True, figsize=(30, 10))
    i = 1

    for company_dict in company_dicts:
        if len(company_dicts) >= 4:
            plt.subplot(
                math.floor((len(company_dicts) / 2)),
                math.ceil((len(company_dicts) / 2)),
                i,
            )
        elif len(company_dicts) == 3:
            plt.subplot(2, 2, i)
        elif len(company_dicts) == 2:
            plt.subplot(2, 1, i)
        elif len(company_dicts) == 1:
            plt.subplot(1, 1, i)
        data = str_to_class(company_dict)
        data["Adj Close"].plot()
        plt.ylabel("Adj Close")
        plt.title(f"Preço de Fechamento de " + list(stocks_list.values())[i - 1])
        i = i + 1


def moving_average(stocks_list, days_list, start, end):
    _, company_list, company_dicts = stock_dicts(stocks_list, start, end)

    for days in days_list:
        for company_dict in company_dicts:
            column_name = f"Média Móvel de {days} dias"
            data = str_to_class(company_dict)
            data[column_name] = data["Adj Close"].rolling(days).mean()

    rotulos = ["Adj Close"]
    for days in days_list:
        rotulos.append("Média Móvel de " + str(days) + " dias")

    plt.figure(constrained_layout=True, figsize=(30, 20))

    i = 1
    for company_dict, company_name in zip(company_dicts, company_list):
        plt.subplot(
            math.floor((len(company_dicts) / 2)), math.ceil((len(company_dicts) / 2)), i
        )
        for rotulo in rotulos:
            data = str_to_class(company_dict)
            data[rotulo].plot()
        plt.title(company_name)
        plt.legend(rotulos)
        plt.legend(loc="upper left")
        i = i + 1


def daily_return(stocks_list, start, end, bar=True, lines=False):
    plt.figure(constrained_layout=True, figsize=(30, 15))

    _, company_list, company_dicts = stock_dicts(stocks_list, start, end)

    for company_dict in company_dicts:
        data = str_to_class(company_dict)
        data["Daily Return"] = data["Adj Close"].pct_change()

    i = 1

    if lines:
        for company_dict, company_name in zip(company_dicts, company_list):
            plt.subplot(
                math.floor((len(company_dicts) / 2)),
                math.ceil((len(company_dicts) / 2)),
                i,
            )
            plt.title(company_name)
            data = str_to_class(company_dict)
            data["Daily Return"].plot(legend=True)
            plt.legend(loc="upper left")
            i = i + 1
    elif bar:
        for company_dict, company_name in zip(company_dicts, company_list):
            plt.subplot(
                math.floor((len(company_dicts) / 2)),
                math.ceil((len(company_dicts) / 2)),
                i,
            )
            plt.title(company_name)
            data = str_to_class(company_dict)
            data["Daily Return"].hist(legend=True)
            plt.legend(loc="upper left")
            i = i + 1


def correlation(stocks_list, start, end, mix_viz=True, heatmap=False):
    # Fazer um DataFrame apenas com os Preços de Fechamento das Empresas
    df_closedmarket = DataReader(list(stocks_list.values()), "yahoo", start, end)[
        "Adj Close"
    ]
    # Retorno Diário de Todas as Ações
    daily_return = df_closedmarket.pct_change()

    if mix_viz:
        # Definir a figura chamando a função PairGrid com o Dataframe:
        other_graphics = sns.PairGrid(daily_return.dropna())
        # Com map_upper(), define-se o que estará no triângulo de cima do gráfico:
        other_graphics.map_upper(plt.scatter, color="purple")
        # Com map_lower(), define-se o que estará no triângulo de baixo do gráfico:
        other_graphics.map_lower(sns.kdeplot, cmap="cool_d")
        # Por fim, a diagonal central, onde permanecerão os histogramas:
        other_graphics.map_diag(plt.hist, bins=30)
    elif heatmap:
        # Incluindo Tabela de Correlação
        sns.heatmap(daily_return.corr(), annot=True, cmap="summer")
    else:
        sns.pairplot(daily_return, kind="reg")


def risk(stocks_list, start, end):
    # Fazer um DataFrame apenas com os Preços de Fechamento das Empresas
    df_closedmarket = DataReader(list(stocks_list.values()), "yahoo", start, end)[
        "Adj Close"
    ]
    # Retorno Diário de Todas as Ações
    daily_return = df_closedmarket.pct_change()

    daily_return_withoutnan = daily_return.dropna()

    area = np.pi * 20

    plt.figure(figsize=(10, 7))
    plt.scatter(daily_return_withoutnan.mean(), daily_return_withoutnan.std(), s=area)
    plt.xlabel("Retorno Esperado")
    plt.ylabel("Risco")

    for label, x, y in zip(
        daily_return_withoutnan.columns,
        daily_return_withoutnan.mean(),
        daily_return_withoutnan.std(),
    ):
        plt.annotate(
            label,
            xy=(x, y),
            xytext=(50, 50),
            textcoords="offset points",
            ha="right",
            va="bottom",
            arrowprops=dict(
                arrowstyle="-", color="blue", connectionstyle="arc3,rad=-0.3"
            ),
        )

## Configuração de Parâmetros

In [ ]:
# Dias para Média Móvel:
days_list = [10, 20, 50]

# Ações a se explorar:
stocks_list = {
    "APPLE": "AAPL",
    "MICROSOFT": "MSFT",
    "BOEING": "BA",
    "AMERICAN AIRLINES": "AAL",
}

# Período de Análise da Ação:
end = datetime.now()
start = datetime(end.year - 1, end.month, end.day)

## Gráficos com Preço de Fechamento ao Longo do Ano Recorrente

In [ ]:
fechamento_ano(stocks_list)

## Gráficos com Média Móvel (Moving Average) por Ação

In [ ]:
moving_average(stocks_list, days_list, start, end)

## Gráficos com média de Retorno Diário por Ação

In [ ]:
daily_return(stocks_list, start, end, bar=True)

## Gráficos com correlação entre os Preços de Fechamento de todas as Ações

In [ ]:
correlation(stocks_list, start, end)

## Risco de Investir em uma Ação

In [ ]:
risk(stocks_list, start, end)

## Previsão do Preço de Fechamento de uma Ação